Check if running direclty on alertfiles improve the processing speed.

In [ ]:
from ampel.lsst.alert.load.ElasticcTrainingsetLoader import ElasticcTrainingsetLoader
from ampel.lsst.alert.ElasticcAlertSupplier import ElasticcAlertSupplier
from ampel.lsst.ingest.LSSTDataPointShaper import LSSTDataPointShaper
from ampel.contrib.hu.t2.T2TabulatorRiseDecline import T2TabulatorRiseDecline
from ampel.contrib.hu.t2.T2TabulatorRiseDecline import T2TabulatorRiseDeclineBase
from ampel.contrib.hu.t2.T2ElasticcRedshiftSampler import T2ElasticcRedshiftSampler
from ampel.dev.DevAmpelContext import DevAmpelContext
from ampel.log.AmpelLogger import AmpelLogger

from ampel.lsst.view.LSSTT2Tabulator import LSSTT2Tabulator
import pandas as pd

In [ ]:
model = 'AGN'
run = 2

In [ ]:
AMPEL_CONF = '/home/jnordin/github/zweilasticc/ampel_conf_d8539b.yaml'

In [ ]:
ctx = DevAmpelContext.load(
    config = AMPEL_CONF,
    db_prefix = 'dumpme'
)
logger = AmpelLogger.get_logger()

In [ ]:
fpath = f'/home/jnordin/data/elasticc/full_elasticc_train/ELASTICC_TRAIN_{model}/ELASTICC_TRAIN_NONIaMODEL0-00{run:02d}'

In [ ]:
# Configure an alert loader
config = {'file_path': fpath}
alertloader = ElasticcTrainingsetLoader(**config)

In [ ]:
# Configure an alert supplier (not sure this is needed?)
config = {'loader': {'unit':'ElasticcTrainingsetLoader', 'config':{'file_path':fpath}}}
supplier = ElasticcAlertSupplier(**config)

In [ ]:
shaper = LSSTDataPointShaper(logger=logger)

In [ ]:
config = {'tabulator':[{'unit':'LSSTT2Tabulator', 'config':{'zp':27.5}}], 'logger': logger}
t2rise = T2TabulatorRiseDecline(**config)

In [ ]:
config = {'logger': logger}
t2z = T2ElasticcRedshiftSampler(**config)

In [ ]:
summary = []
hostinfo = {}

In [ ]:
for k, alert in enumerate(supplier):
    if k>1000:
        break
    
    stock = alert.stock
    dps = shaper.process(alert.datapoints, stock)
    
    risdec = t2rise.process({}, dps )
    # Do not bloat with non-detection.
    if not risdec['success']:
        continue
        
    
    summary.append( risdec )
    
    if not stock in hostinfo:
        dpobj = [dp for dp in dps if 'LSST_OBJ' in dp['tag']]
        hostinfo[stock] = t2z.process(dpobj[0] )

    if len(summary) % 100==0:
        print(len(summary))
        print(summary[-1])

In [ ]:
print(len(summary))

In [ ]:
print(len(hostinfo))

In [ ]:
dfh = pd.DataFrame.from_dict(hostinfo, orient='index')

In [ ]:
outf = f'hostinfo_{model}_{run:02d}.csv'

In [ ]:
dfh.to_csv(outf)

In [ ]:
dfs = pd.DataFrame.from_dict(summary)

In [ ]:
outs = f'risedec_{model}_{run:02d}.csv'

In [ ]:
dfs.to_csv(outs)